In [4]:
from datasets import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import pandas as pd
import torch
import os 
from sklearn.preprocessing import LabelEncoder
# Change to the parent directory of the notebook

os.chdir('..')
os.chdir('..')

In [5]:
import torch

In [6]:
torch.cuda.is_available()

True

In [7]:
df1 = pd.read_csv("Data/knowledge_base_data/explainations_CoT_Hermes_partial.csv") #, index_col=0) # first 9000

In [8]:
df2 = pd.read_csv("Data/knowledge_base_data/explainations_CoT_Hermes_partial_2.csv", index_col=1) #9000-10000

In [9]:
df3 = pd.read_csv("Data/knowledge_base_data/explainations_CoT_Hermes_partial_3.csv", index_col=1) # 15k to 19k

In [10]:
df4 = pd.read_csv("Data/knowledge_base_data/explainations_CoT_Hermes_partial_4.csv", index_col=1) # 11k to 15k 

In [11]:
df_combined = pd.concat([df1, df2, df3, df4], ignore_index=False) 

In [17]:
# Extract unique words from the 'target' column
unique_words = set()

df_combined['target'].dropna().apply(lambda x: unique_words.update(x.split(', ')))

# Convert to a sorted list (optional)
unique_words_list = sorted(unique_words)

# Print or return the list
print(unique_words_list)


['African', 'Arab', 'Asexual', 'Asian', 'Bisexual', 'Buddhism', 'Caucasian', 'Christian', 'Disability', 'Economic', 'Heterosexual', 'Hindu', 'Hispanic', 'Homosexual', 'Indian', 'Indigenous', 'Islam', 'Jewish', 'Men', 'Minority', 'None', 'Nonreligious', 'Other', 'Refugee', 'Women']


In [14]:
# Splitting the multi-category entries and getting unique words
unique_words = set()
df_combined['target'].dropna().apply(lambda x: unique_words.update(x.split(', ')))

# Count of unique words
len(unique_words)


25

In [30]:
# df = pd.read_csv("Data/dataset.csv")

In [ ]:

# # Load dataset
# df = pd.read_csv("Data/dataset.csv")
# df = df[['tweet_text', 'label']]

# # Assuming your label column is called 'label'
# label_encoder = LabelEncoder()
# df['labels'] = label_encoder.fit_transform(df['label'])
# df = df[['tweet_text', 'labels']]

In [21]:


# # Split dataset
# train_df, val_df = train_test_split(df, test_size=0.2)

# # Convert to Hugging Face Dataset
# train_dataset = Dataset.from_pandas(train_df)
# val_dataset = Dataset.from_pandas(val_df)

# # Initialize tokenizer and model
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# # Tokenize data
# # Tokenize data with max_length
# def tokenize_function(examples):
#     return tokenizer(
#         examples['tweet_text'], 
#         padding='max_length',  # Pad to the max length (you can specify a max_length if needed)
#         truncation=True,        # Truncate if the sequence is too long
#         max_length=128          # Set max_length as per your requirement
#     )

# train_dataset = train_dataset.map(tokenize_function, batched=True)
# val_dataset = val_dataset.map(tokenize_function, batched=True)

# # Rename the 'label' column to 'labels' to match the expected column name
# # train_dataset = train_dataset.rename_column("label", "labels")
# # val_dataset = val_dataset.rename_column("label", "labels")

# # Set the device to GPU 1 manually (cuda:1)
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print(device)
# model.to(device)  # Move model to the selected device


In [22]:

# import numpy as np
# from sklearn.metrics import accuracy_score, f1_score

# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
#     acc = accuracy_score(labels, predictions)
#     f1 = f1_score(labels, predictions, average='weighted')
#     return {"accuracy": acc, "f1": f1}
    
# training_args = TrainingArguments(
#     output_dir='./results',  # Directory for final model
#     evaluation_strategy="steps",  # Evaluate at end of each epoch
#     eval_steps=50,
#     per_device_train_batch_size=24,
#     per_device_eval_batch_size=24,
#     num_train_epochs=3,
#     logging_dir='./logs',  # Directory for logs
#     logging_strategy="steps",  # Log training progress
#     logging_steps=10,  # Log every 10 steps
#     save_strategy="no",  # Disable automatic checkpoint saving
#     save_total_limit=0,  # Ensure no checkpoints are kept
#     no_cuda=False,  # Use GPU if available,
#     # learning_rate=2e-5,
#     # max_grad_norm=1.0,
#     # fp16=False,
#     # report_to=["console"]
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     compute_metrics=compute_metrics,
#     tokenizer=tokenizer,
#     # debug=True  # Enable debugging
# )

# # Train the model
# trainer.train()

# # Save the final model
# trainer.save_model("./final_model")  # Saves only at the end


In [23]:
# print(train_dataset['labels'][:10])  # Check if they look correct


In [24]:
# print(train_dataset[0])
# print(val_dataset[0])


In [25]:
# from sklearn.metrics import classification_report

# # Get predictions
# predictions = trainer.predict(val_dataset)
# y_pred = predictions.predictions.argmax(axis=-1)
# y_true = val_df['label'].values  # True labels

# # Print classification report
# print(classification_report(y_true, y_pred))


In [2]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
import os

os.chdir('..')
os.chdir('..')
# Load dataset
df = pd.read_csv("Data/dataset.csv")
df = df[['tweet_text', 'label']]

# Encode labels
label_encoder = LabelEncoder()
df['labels'] = label_encoder.fit_transform(df['label'])
df = df[['tweet_text', 'labels']]

# Split dataset into train and validation
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['tweet_text'].tolist(), df['labels'].tolist(), test_size=0.1, random_state=42
)

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True, max_length=512)

# Convert data into Hugging Face Dataset format
dataset = Dataset.from_dict({
    "text": train_texts + val_texts,
    "label": train_labels + val_labels
})

dataset = dataset.map(tokenize_function, batched=True)
train_dataset = dataset.select(range(len(train_texts)))
val_dataset = dataset.select(range(len(train_texts), len(train_texts) + len(val_texts)))

# Load BERT model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_encoder.classes_))

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    return {"accuracy": acc, "f1": f1}

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="steps",
    eval_steps=50,
    
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
    num_train_epochs=3,
    logging_dir='./logs',
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="no",
    save_total_limit=0,
    no_cuda=False,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer)
)

# Train model
trainer.train()

# Save final model
trainer.save_model("./final_model")


Map:   0%|          | 0/20148 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\MachineLearning\AICU\recommendation_system\.venv\Lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\usama\AppData\Local\Temp\ipykernel_49972\935123486.py:71: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss,Accuracy,F1
50,0.881100,0.929643,0.582630,0.539938
100,0.905700,0.888768,0.605955,0.591407
150,0.776200,0.827797,0.650124,0.641985
200,0.844200,0.832451,0.620347,0.588882
250,0.853300,0.839361,0.632258,0.620586
300,0.742900,0.798361,0.655087,0.629946
350,0.720600,0.804100,0.666005,0.655984
400,0.790300,0.782104,0.671464,0.672194
450,0.818900,0.779998,0.672457,0.673772
500,0.775700,0.767702,0.673945,0.668762


KeyboardInterrupt: 

In [3]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased', language='english')
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))

# Prepare the dataset for BERT
def prepare_data(examples):
    encodings = tokenizer(
        examples['tweet_text'], 
        truncation=True, 
        padding=True,
        return_tensors='pt'
    )
    labels = torch.tensor([label_encoder.transform(example) for example in examples['label']], dtype=torch.long)
    return {
        'input_ids': encodings['input_ids'].flatten(),
        'attention_mask': encodings['attention_mask'].flatten(),
        'labels': labels
    }

# Create dataset from dataframe
dataset = Dataset.from_dict(df.to_dict())
train_dataset = dataset.train_test_split(train_size=0.8, shuffle=True).values()

# Map the dataset to tensors
train_dataset = train_dataset.map(prepare_data, batched=True)
val_dataset = val_dataset.map(prepare_data, batched=True)

# Train the model
trainer.train()

# Save the final model and tokenizer
model.save_pretrained("./final_model")
tokenizer.save_pretrained("./final_tokenizer")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


AttributeError: 'dict_values' object has no attribute 'map'

In [ ]:
import os

os.chdir('..')
os.chdir('..')
# Load dataset
df = pd.read_csv("Data/dataset.csv")
df = df[['tweet_text', 'label']]

# Encode labels
label_encoder = LabelEncoder()
df['labels'] = label_encoder.fit_transform(df['label'])
df = df[['tweet_text', 'labels']]

# Split dataset into train and validation
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['tweet_text'].tolist(), df['labels'].tolist(), test_size=0.1, random_state=42
)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)


small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)
trainer.train()

In [17]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files="Data/dataset.csv")

split_datasets = dataset["train"].train_test_split(test_size=0.1, seed=42)


In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'post_id', 'tweet_text', 'key_features', 'target', 'label', 'annotator_1_label', 'annotator_1_target', 'annotator_2_label', 'annotator_2_target', 'annotator_3_label', 'annotator_3_target'],
        num_rows: 20148
    })
})

In [18]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["tweet_text"], padding="max_length", truncation=True)

tokenized_datasets = split_datasets.map(tokenize_function, batched=True)


Map:   0%|          | 0/18133 [00:00<?, ? examples/s]

Map:   0%|          | 0/2015 [00:00<?, ? examples/s]

In [19]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))


In [20]:
len(set(dataset["train"]["label"]))

3

In [21]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "google-bert/bert-base-cased",
    num_labels=len(set(dataset["train"]["label"]))  # Adjust based on your labels
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="test_trainer",
    # evaluation_strategy="epoch",
    # save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="steps",
    eval_steps=50,
)


In [23]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)
trainer.train()

ValueError: too many dimensions 'str'

## bert

In [10]:
df_combined

,Row Number,Input,tweet_text,key_features,label,target,Response,Time Taken (s),Unnamed: 0
0,1.0,"{'post_id': '1179055004553900032_twitter', 'tw...",i dont think im getting my baby them white 9 h...,[],normal,NaN,The text appears to be a personal statement ab...,39.40,NaN
1,2.0,"{'post_id': '1179063826874032128_twitter', 'tw...",we cannot continue calling ourselves feminists...,[],normal,NaN,The text falls under the category of normal. H...,18.88,NaN
2,3.0,"{'post_id': '1178793830532956161_twitter', 'tw...",nawt yall niggers ignoring me,[],normal,"African, None",This text cannot be classified as normal. The ...,13.28,NaN
3,4.0,"{'post_id': '1179088797964763136_twitter', 'tw...",<user> i am bit confused coz chinese ppl can n...,"['ching', 'chong']",hatespeech,Asian,The text is classified as hate speech due to t...,27.10,NaN
4,5.0,"{'post_id': '1179085312976445440_twitter', 'tw...",this bitch in whataburger eating a burger with...,"['bitch', 'i', 'hate', 'white', 'bitches']",hatespeech,"Caucasian, Women",**Hate Speech**:\r\n\r\nThe tweet explicitly e...,18.15,NaN
...,...,...,...,...,...,...,...,...,...
15995,NaN,"{'post_id': '18011618_gab', 'tweet_text': 'mac...",macron is going to demand that we take scumbag...,"['scumbag', 'refugees']",offensive,Refugee,The text is classified as offensive due to the...,24.93,995.0
15996,NaN,"{'post_id': '18012075_gab', 'tweet_text': 'the...",the national health serivces of the uk making ...,[],normal,"Men, None, Women",The text falls into the neutral category due t...,17.51,996.0
15997,NaN,"{'post_id': '18013591_gab', 'tweet_text': 'and...",and obama hung out with white ppl in martha vi...,[],normal,"African, Caucasian, None",The text falls into the normal category due to...,32.13,997.0
15998,NaN,"{'post_id': '18015405_gab', 'tweet_text': 'it ...",it is the white man fault the nigger raped me ...,"['white', 'man', 'fault', 'the', 'nigger', 'ra...",hatespeech,"African, Hispanic, None, Women",The tweet text is classified as hate speech du...,25.79,998.0


In [65]:
df = pd.read_csv("Data/dataset.csv", index_col=0)

In [16]:
# TRAIN_DF = pd.read_csv("Data/TRAIN_DF.csv", index_col=0)
TEST_DF = pd.read_csv("Data/TEST_DF.csv", index_col=0)

In [67]:
TRAIN_DF = df.loc[~df.index.isin(TEST_DF.index)]

In [10]:
# from exp datasets
TEST_DF = pd.read_csv("Data/TEST_DF.csv", index_col=0)
TRAIN_DF = df_combined
TRAIN_DF = TRAIN_DF.loc[~TRAIN_DF.index.isin(TEST_DF.index)]

In [24]:
TRAIN_DF['Response'].isna().sum()

np.int64(0)

In [25]:
TRAIN_DF['tweet_text'] = TRAIN_DF['tweet_text'] + "\nContext:" + TRAIN_DF['Response']

In [26]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt


class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        # Convert to lists to ensure indexing works correctly
        self.texts = texts.tolist() if hasattr(texts, 'tolist') else list(texts)
        self.labels = labels.tolist() if hasattr(labels, 'tolist') else list(labels)
        self.tokenizer = tokenizer
        self.max_length = max_length

        # Verify data integrity
        assert len(self.texts) == len(
            self.labels), "Texts and labels must have the same length"

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        if idx >= len(self.texts):
            raise IndexError(
                f"Index {idx} out of bounds for dataset of size {len(self.texts)}")

        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


def train_model(model, train_loader, val_loader, device, le, epochs=3):
    optimizer = AdamW(model.parameters(), lr=2e-5)
    best_accuracy = 0

    for epoch in range(epochs):
        print(f'\nEpoch {epoch + 1}/{epochs}')

        # Training
        model.train()
        train_loss = 0
        train_steps = 0

        #for batch_idx, batch in enumerate(tqdm(train_loader, desc='Training')):
        for batch_idx, batch in enumerate(train_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )

            loss = outputs.loss
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_steps += 1
            
            # Print every 50 steps
            if (batch_idx + 1) % 50 == 0:
                avg_train_loss = train_loss / train_steps
                print(f'Step {batch_idx + 1} - Average training loss: {avg_train_loss:.4f}')

        avg_train_loss = train_loss / train_steps
        print(f'Average training loss: {avg_train_loss:.4f}')

        # Validation
        model.eval()
        val_loss = 0
        val_steps = 0
        all_predictions = []
        all_true_labels = []

        with torch.no_grad():
            for batch in tqdm(val_loader, desc='Validation'):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                outputs = model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    labels=labels
                )

                loss = outputs.loss
                val_loss += loss.item()

                predictions = torch.argmax(outputs.logits, dim=1)
                all_predictions.extend(predictions.cpu().numpy())
                all_true_labels.extend(labels.cpu().numpy())
                val_steps += 1

        # Calculate metrics
        avg_val_loss = val_loss / val_steps
        accuracy = (np.array(all_predictions) ==
                    np.array(all_true_labels)).mean()

        print(f'Average validation loss: {avg_val_loss:.4f}')
        print(f'Validation accuracy: {accuracy:.4f}')
        print('\nClassification Report:')
        print(classification_report(all_true_labels, all_predictions,
                                    target_names=le.classes_))

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            torch.save(model.state_dict(), 'best_model.pt')
            print(f'New best model saved with accuracy: {accuracy:.4f}')



In [14]:
TRAIN_DF = TRAIN_DF[:18840]

In [31]:
# TRAIN_DF.to_csv("Data/TRAIN_DF.csv")

In [15]:
TRAIN_DF.shape

(17117, 9)

In [16]:
TEST_DF.shape

(1884, 17)

In [17]:
TEST_DF['label'].value_counts()

label
hatespeech    628
normal        628
offensive     628
Name: count, dtype: int64

In [27]:
# Modify the main function to use TRAIN_DF for training and TEST_DF for testing
def main():
    try:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        # Load the training and test data
        train_df = TRAIN_DF # pd.read_csv('Data/TRAIN_DF.csv')
        test_df = TEST_DF # pd.read_csv('Data/TEST_DF.csv')

        # Data validation
        required_columns = ['tweet_text', 'label']
        if not all(col in train_df.columns for col in required_columns):
            raise ValueError(f"Dataset must contain columns: {required_columns}")
     
        train_df = train_df.dropna(subset=['tweet_text', 'label'])
        test_df = test_df.dropna(subset=['tweet_text', 'label'])

        # Reset index after dropping NaN values
        train_df = train_df.reset_index(drop=True)
        test_df = test_df.reset_index(drop=True)

        # Convert labels to numerical values
        le = LabelEncoder()
        train_df['label'] = le.fit_transform(train_df['label'])
        test_df['label'] = le.transform(test_df['label'])  # Use the same encoder for test data

        # Split the dataset into train and validation sets
        print("\nSplitting training dataset...")
        train_texts, val_texts, train_labels, val_labels = train_test_split(
            train_df['tweet_text'],
            train_df['label'],
            test_size=0.15,
            random_state=42,
            stratify=train_df['label']
        )

        # Initialize tokenizer and model
        print("Initializing BERT model and tokenizer...")
        tokenizer = BertTokenizer.from_pretrained('google-bert/bert-large-uncased')
        num_labels = len(le.classes_)
        model = BertForSequenceClassification.from_pretrained(
            'google-bert/bert-large-uncased',
            num_labels=num_labels
        )

        # Create datasets
        train_dataset = CustomDataset(train_texts, train_labels, tokenizer)
        val_dataset = CustomDataset(val_texts, val_labels, tokenizer)

        # Create dataloaders
        train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=64)

        # Move model to GPU if available
        model.to(device)

        # Train the model
        train_model(model, train_loader, val_loader, device, le, 4)

        # Testing on TEST_DF (used only for testing at the end)
        print("\nEvaluating model on the test dataset...")
        test_texts = test_df['tweet_text']
        test_labels = test_df['label']

        test_dataset = CustomDataset(test_texts, test_labels, tokenizer)
        test_loader = DataLoader(test_dataset, batch_size=64)

        # Evaluate the model on test data
        model.eval()
        all_predictions = []
        all_true_labels = []
        test_loss = 0
        test_steps = 0

        with torch.no_grad():
            for batch in tqdm(test_loader, desc='Testing'):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                test_loss += loss.item()

                predictions = torch.argmax(outputs.logits, dim=1)
                all_predictions.extend(predictions.cpu().numpy())
                all_true_labels.extend(labels.cpu().numpy())
                test_steps += 1

        avg_test_loss = test_loss / test_steps
        print(f'Average test loss: {avg_test_loss:.4f}')
        print('\nTest Classification Report:')
        print(classification_report(all_true_labels, all_predictions, target_names=le.classes_))

        # Save the final model and tokenizer
        model.save_pretrained('fine_tuned_bert_with_explainations')
        tokenizer.save_pretrained('fine_tuned_bert_with_explainations')
        np.save('label_classes.npy', le.classes_)

    except Exception as e:
        print(f"\nError occurred: {str(e)}")
        print("\nFull error details:")
        import traceback
        traceback.print_exc()


In [28]:
# def main():
#     try:
        
#         device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#         # Use TRAIN_DF and TEST_DF instead of reading from the file
#         train_df = TRAIN_DF
#         test_df = TEST_DF

#         # Data validation
#         required_columns = ['tweet_text', 'label']
#         if not all(col in train_df.columns for col in required_columns):
#             raise ValueError(f"Dataset must contain columns: {required_columns}")

#         train_df = train_df.dropna(subset=['tweet_text', 'label'])
#         test_df = test_df.dropna(subset=['tweet_text', 'label'])

#         # Reset index after dropping NaN values
#         train_df = train_df.reset_index(drop=True)
#         test_df = test_df.reset_index(drop=True)

#         # Convert labels to numerical values
#         le = LabelEncoder()
#         train_df['label'] = le.fit_transform(train_df['label'])
#         test_df['label'] = le.transform(test_df['label'])  # Use the same encoder for test data

#         # Print class distribution
#         print("\nTraining set class distribution:")
#         for i, label in enumerate(le.classes_):
#             count = len(train_df[train_df['label'] == i])
#             print(f"{label}: {count} samples")

#         # Create datasets
#         print("\nCreating datasets...")
#         train_texts, train_labels = train_df['tweet_text'], train_df['label']
#         test_texts, test_labels = test_df['tweet_text'], test_df['label']

#         # Initialize tokenizer and model
#         print("Initializing BERT model and tokenizer...")
#         tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#         num_labels = len(le.classes_)
#         model = BertForSequenceClassification.from_pretrained(
#             'bert-base-uncased',
#             num_labels=num_labels
#         )

#         # Create datasets
#         train_dataset = CustomDataset(train_texts, train_labels, tokenizer)
#         test_dataset = CustomDataset(test_texts, test_labels, tokenizer)

#         # Create dataloaders
#         train_loader = DataLoader(train_dataset, batch_size=50, shuffle=True)
#         test_loader = DataLoader(test_dataset, batch_size=50)

#         # Move model to GPU if available
#         model.to(device)

#         # Train the model
#         train_model(model, train_loader, test_loader, device, le)

#         # Save the final model and tokenizer
#         model.save_pretrained('fine_tuned_bert')
#         tokenizer.save_pretrained('fine_tuned_bert')
#         np.save('label_classes.npy', le.classes_)

#     except Exception as e:
#         print(f"\nError occurred: {str(e)}")
#         print("\nFull error details:")
#         import traceback
#         traceback.print_exc()


In [29]:

# def main():
#     try:
        
#         device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#         df = pd.read_csv('Data/dataset.csv')

#         # Data validation
#         required_columns = ['tweet_text', 'label']
#         if not all(col in df.columns for col in required_columns):
#             raise ValueError(
#                 f"Dataset must contain columns: {required_columns}")
     
#         df = df.dropna(subset=['tweet_text', 'label'])

#         # Reset index after dropping NaN values
#         df = df.reset_index(drop=True)

#         # Convert labels to numerical values
#         le = LabelEncoder()
#         df['label'] = le.fit_transform(df['label'])

#         # Print class distribution
       
#         for i, label in enumerate(le.classes_):
#             count = len(df[df['label'] == i])
#             print(f"{label}: {count} samples")

#         # Split the dataset
#         print("\nSplitting dataset...")
#         train_texts, val_texts, train_labels, val_labels = train_test_split(
#             df['tweet_text'],
#             df['label'],
#             test_size=0.2,
#             random_state=42,
#             stratify=df['label']
#         )

#         # Initialize tokenizer and model
#         print("Initializing BERT model and tokenizer...")
#         tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#         num_labels = 3
#         model = BertForSequenceClassification.from_pretrained(
#             'bert-base-uncased',
#             num_labels=num_labels
#         )

#         # Create datasets
#         train_dataset = CustomDataset(train_texts, train_labels, tokenizer)
#         val_dataset = CustomDataset(val_texts, val_labels, tokenizer)

#         # Create dataloaders
#         train_loader = DataLoader(
#             train_dataset,
#             batch_size=24,
#             shuffle=True
#         )
#         val_loader = DataLoader(
#             val_dataset,
#             batch_size=24
#         )

#         # Move model to GPU if available
#         model.to(device)

#         # Train the model

#         train_model(model, train_loader, val_loader, device, le)

#         # Save the final model and tokenizer

#         model.save_pretrained('fine_tuned_bert')
#         tokenizer.save_pretrained('fine_tuned_bert')
#         np.save('label_classes.npy', le.classes_)

   

#     except Exception as e:
#         print(f"\nError occurred: {str(e)}")
#         print("\nFull error details:")
#         import traceback
#         traceback.print_exc()


In [30]:
# with exp
main()


Splitting training dataset...
Initializing BERT model and tokenizer...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Python\py312env\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



Epoch 1/4
Step 50 - Average training loss: 0.4917
Step 100 - Average training loss: 0.3156
Step 150 - Average training loss: 0.2497
Step 200 - Average training loss: 0.2107
Average training loss: 0.1954


Validation: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 41/41 [00:19<00:00,  2.08it/s]


Average validation loss: 0.0790
Validation accuracy: 0.9778

Classification Report:
              precision    recall  f1-score   support

  hatespeech       0.97      0.98      0.98       794
      normal       0.99      0.97      0.98      1076
   offensive       0.98      0.98      0.98       698

    accuracy                           0.98      2568
   macro avg       0.98      0.98      0.98      2568
weighted avg       0.98      0.98      0.98      2568

New best model saved with accuracy: 0.9778

Epoch 2/4
Step 50 - Average training loss: 0.0741
Step 100 - Average training loss: 0.0686
Step 150 - Average training loss: 0.0652
Step 200 - Average training loss: 0.0691
Average training loss: 0.0690


...

Average validation loss: 0.0653
Validation accuracy: 0.9836

Classification Report:
              precision    recall  f1-score   support

  hatespeech       0.98      0.99      0.98       794
      normal       0.99      0.98      0.98      1076
   offensive       0.99      0.98      0.99       698

    accuracy                           0.98      2568
   macro avg       0.98      0.98      0.98      2568
weighted avg       0.98      0.98      0.98      2568

New best model saved with accuracy: 0.9836

Epoch 4/4
Step 50 - Average training loss: 0.0418
Step 100 - Average training loss: 0.0440
Step 150 - Average training loss: 0.0425
Step 200 - Average training loss: 0.0403
Average training loss: 0.0384


Validation: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 41/41 [00:20<00:00,  2.04it/s]


Average validation loss: 0.0603
Validation accuracy: 0.9813

Classification Report:
              precision    recall  f1-score   support

  hatespeech       0.98      0.98      0.98       794
      normal       0.98      0.98      0.98      1076
   offensive       0.99      0.98      0.98       698

    accuracy                           0.98      2568
   macro avg       0.98      0.98      0.98      2568
weighted avg       0.98      0.98      0.98      2568


Evaluating model on the test dataset...


Testing: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:10<00:00,  2.93it/s]


Average test loss: 2.2874

Test Classification Report:
              precision    recall  f1-score   support

  hatespeech       0.65      0.02      0.04       628
      normal       0.34      0.98      0.51       628
   offensive       0.30      0.03      0.05       628

    accuracy                           0.34      1884
   macro avg       0.43      0.34      0.20      1884
weighted avg       0.43      0.34      0.20      1884



In [22]:
# without exp
main()


Splitting training dataset...
Initializing BERT model and tokenizer...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Python\py312env\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



Epoch 1/4
Step 50 - Average training loss: 1.0529
Step 100 - Average training loss: 0.9960
Step 150 - Average training loss: 0.9442
Step 200 - Average training loss: 0.9174
Average training loss: 0.9053


Validation: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 41/41 [00:13<00:00,  2.97it/s]


Average validation loss: 0.7748
Validation accuracy: 0.6655

Classification Report:
              precision    recall  f1-score   support

  hatespeech       0.76      0.71      0.74       794
      normal       0.73      0.73      0.73      1076
   offensive       0.48      0.52      0.50       698

    accuracy                           0.67      2568
   macro avg       0.66      0.65      0.65      2568
weighted avg       0.67      0.67      0.67      2568

New best model saved with accuracy: 0.6655

Epoch 2/4
Step 50 - Average training loss: 0.7648
Step 100 - Average training loss: 0.7619
Step 150 - Average training loss: 0.7582
Step 200 - Average training loss: 0.7515
Average training loss: 0.7497


...

Average validation loss: 0.7078
Validation accuracy: 0.6990

Classification Report:
              precision    recall  f1-score   support

  hatespeech       0.77      0.74      0.76       794
      normal       0.69      0.87      0.77      1076
   offensive       0.61      0.38      0.47       698

    accuracy                           0.70      2568
   macro avg       0.69      0.67      0.67      2568
weighted avg       0.69      0.70      0.68      2568

New best model saved with accuracy: 0.6990

Epoch 4/4
Step 50 - Average training loss: 0.6182
Step 100 - Average training loss: 0.6118
Step 150 - Average training loss: 0.6062
Step 200 - Average training loss: 0.6105
Average training loss: 0.6104


Validation: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 41/41 [00:13<00:00,  2.95it/s]


Average validation loss: 0.7147
Validation accuracy: 0.7040

Classification Report:
              precision    recall  f1-score   support

  hatespeech       0.73      0.83      0.78       794
      normal       0.74      0.80      0.77      1076
   offensive       0.57      0.41      0.48       698

    accuracy                           0.70      2568
   macro avg       0.68      0.68      0.68      2568
weighted avg       0.69      0.70      0.69      2568

New best model saved with accuracy: 0.7040

Evaluating model on the test dataset...


Testing: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:10<00:00,  2.95it/s]


Average test loss: 0.9052

Test Classification Report:
              precision    recall  f1-score   support

  hatespeech       0.78      0.69      0.73       628
      normal       0.55      0.79      0.65       628
   offensive       0.61      0.41      0.49       628

    accuracy                           0.63      1884
   macro avg       0.65      0.63      0.62      1884
weighted avg       0.65      0.63      0.62      1884



In [23]:
main()


Splitting training dataset...
Initializing BERT model and tokenizer...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Python\py312env\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



Epoch 1/3
Step 50 - Average training loss: 1.0456
Step 100 - Average training loss: 0.9785
Step 150 - Average training loss: 0.9376
Step 200 - Average training loss: 0.9173
Average training loss: 0.9048


Validation: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 58/58 [00:19<00:00,  2.96it/s]


Average validation loss: 0.8169
Validation accuracy: 0.6356

Classification Report:
              precision    recall  f1-score   support

  hatespeech       0.61      0.80      0.69      1103
      normal       0.67      0.72      0.69      1443
   offensive       0.61      0.36      0.45      1107

    accuracy                           0.64      3653
   macro avg       0.63      0.63      0.61      3653
weighted avg       0.63      0.64      0.62      3653

New best model saved with accuracy: 0.6356

Epoch 2/3
Step 50 - Average training loss: 0.7718
Step 100 - Average training loss: 0.7802
Step 150 - Average training loss: 0.7672
Step 200 - Average training loss: 0.7676
Average training loss: 0.7679


...

Average validation loss: 0.7628
Validation accuracy: 0.6633

Classification Report:
              precision    recall  f1-score   support

  hatespeech       0.78      0.68      0.72      1103
      normal       0.64      0.82      0.72      1443
   offensive       0.58      0.44      0.50      1107

    accuracy                           0.66      3653
   macro avg       0.67      0.65      0.65      3653
weighted avg       0.66      0.66      0.66      3653

New best model saved with accuracy: 0.6633

Epoch 3/3
Step 50 - Average training loss: 0.6704
Step 100 - Average training loss: 0.6705
Step 150 - Average training loss: 0.6665
Step 200 - Average training loss: 0.6667
Average training loss: 0.6703


Validation: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 58/58 [00:19<00:00,  2.92it/s]


Average validation loss: 0.7528
Validation accuracy: 0.6729

Classification Report:
              precision    recall  f1-score   support

  hatespeech       0.77      0.70      0.73      1103
      normal       0.69      0.76      0.72      1443
   offensive       0.56      0.54      0.55      1107

    accuracy                           0.67      3653
   macro avg       0.67      0.66      0.67      3653
weighted avg       0.67      0.67      0.67      3653

New best model saved with accuracy: 0.6729

Evaluating model on the test dataset...


Testing: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:10<00:00,  2.95it/s]


Average test loss: 0.6538

Test Classification Report:
              precision    recall  f1-score   support

  hatespeech       0.90      0.73      0.81       628
      normal       0.65      0.88      0.75       628
   offensive       0.72      0.60      0.66       628

    accuracy                           0.74      1884
   macro avg       0.76      0.74      0.74      1884
weighted avg       0.76      0.74      0.74      1884



In [8]:
main()


Splitting training dataset...
Initializing BERT model and tokenizer...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\MachineLearning\AICU\recommendation_system\.venv\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



Epoch 1/6
Step 50 - Average training loss: 0.9396
Step 100 - Average training loss: 0.8085
Average training loss: 0.7918


Validation: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:09<00:00,  2.93it/s]


Average validation loss: 0.5998
Validation accuracy: 0.7667

Classification Report:
              precision    recall  f1-score   support

  hatespeech       0.78      0.87      0.82       682
      normal       0.81      0.84      0.82       857
   offensive       0.44      0.27      0.34       261

    accuracy                           0.77      1800
   macro avg       0.68      0.66      0.66      1800
weighted avg       0.75      0.77      0.75      1800

New best model saved with accuracy: 0.7667

Epoch 2/6
Step 50 - Average training loss: 0.5659
Step 100 - Average training loss: 0.5595
Average training loss: 0.5496


...

Average validation loss: 0.6423
Validation accuracy: 0.7728

Classification Report:
              precision    recall  f1-score   support

  hatespeech       0.79      0.86      0.82       682
      normal       0.82      0.85      0.83       857
   offensive       0.47      0.30      0.37       261

    accuracy                           0.77      1800
   macro avg       0.69      0.67      0.67      1800
weighted avg       0.76      0.77      0.76      1800


Epoch 6/6
Step 50 - Average training loss: 0.2045
Step 100 - Average training loss: 0.1932
Average training loss: 0.1939


Validation: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:09<00:00,  3.01it/s]


Average validation loss: 0.8712
Validation accuracy: 0.7678

Classification Report:
              precision    recall  f1-score   support

  hatespeech       0.78      0.88      0.83       682
      normal       0.85      0.80      0.82       857
   offensive       0.44      0.38      0.41       261

    accuracy                           0.77      1800
   macro avg       0.69      0.69      0.69      1800
weighted avg       0.76      0.77      0.76      1800


Evaluating model on the test dataset...


Testing: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:10<00:00,  2.98it/s]


Average test loss: 1.8070

Test Classification Report:
              precision    recall  f1-score   support

  hatespeech       0.81      0.66      0.72       628
      normal       0.49      0.88      0.63       628
   offensive       0.62      0.25      0.35       628

    accuracy                           0.59      1884
   macro avg       0.64      0.59      0.57      1884
weighted avg       0.64      0.59      0.57      1884



In [13]:
main()


Splitting training dataset...
Initializing BERT model and tokenizer...


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

Average validation loss: 0.9329
Validation accuracy: 0.7561

Classification Report:
              precision    recall  f1-score   support

  hatespeech       0.78      0.85      0.81       682
      normal       0.85      0.80      0.82       857
   offensive       0.40      0.39      0.39       261

    accuracy                           0.76      1800
   macro avg       0.67      0.68      0.68      1800
weighted avg       0.76      0.76      0.76      1800


Epoch 10/10
Step 50 - Average training loss: 0.1869
Step 100 - Average training loss: 0.2028
Average training loss: 0.2098


Validation: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:11<00:00,  2.63it/s]


Average validation loss: 0.7476
Validation accuracy: 0.7678

Classification Report:
              precision    recall  f1-score   support

  hatespeech       0.74      0.91      0.82       682
      normal       0.87      0.77      0.82       857
   offensive       0.49      0.37      0.42       261

    accuracy                           0.77      1800
   macro avg       0.70      0.69      0.69      1800
weighted avg       0.77      0.77      0.76      1800


Evaluating model on the test dataset...


Testing: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:11<00:00,  2.60it/s]


Average test loss: 1.7452

Test Classification Report:
              precision    recall  f1-score   support

  hatespeech       0.75      0.68      0.71       628
      normal       0.50      0.84      0.63       628
   offensive       0.65      0.26      0.37       628

    accuracy                           0.59      1884
   macro avg       0.63      0.59      0.57      1884
weighted avg       0.63      0.59      0.57      1884



In [6]:
# with explainations
main()


Splitting training dataset...
Initializing BERT model and tokenizer...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\MachineLearning\AICU\recommendation_system\.venv\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



Epoch 1/6
Step 50 - Average training loss: 0.9250
Step 100 - Average training loss: 0.8059
Average training loss: 0.7840


Validation: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:09<00:00,  3.08it/s]


Average validation loss: 0.5891
Validation accuracy: 0.7722

Classification Report:
              precision    recall  f1-score   support

  hatespeech       0.78      0.90      0.83       682
      normal       0.80      0.86      0.83       857
   offensive       0.47      0.16      0.24       261

    accuracy                           0.77      1800
   macro avg       0.68      0.64      0.63      1800
weighted avg       0.74      0.77      0.74      1800

New best model saved with accuracy: 0.7722

Epoch 2/6
Step 50 - Average training loss: 0.5489
Step 100 - Average training loss: 0.5241
Average training loss: 0.5244


...

Average validation loss: 0.8168
Validation accuracy: 0.7506

Classification Report:
              precision    recall  f1-score   support

  hatespeech       0.77      0.87      0.82       682
      normal       0.87      0.76      0.81       857
   offensive       0.38      0.42      0.40       261

    accuracy                           0.75      1800
   macro avg       0.67      0.68      0.68      1800
weighted avg       0.76      0.75      0.75      1800


Epoch 6/6
Step 50 - Average training loss: 0.1158
Step 100 - Average training loss: 0.1339
Average training loss: 0.1361


Validation: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:09<00:00,  3.00it/s]


Average validation loss: 0.9502
Validation accuracy: 0.7528

Classification Report:
              precision    recall  f1-score   support

  hatespeech       0.81      0.82      0.81       682
      normal       0.86      0.78      0.82       857
   offensive       0.38      0.48      0.42       261

    accuracy                           0.75      1800
   macro avg       0.68      0.69      0.69      1800
weighted avg       0.77      0.75      0.76      1800


Evaluating model on the test dataset...


Testing: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:10<00:00,  2.97it/s]


Average test loss: 1.9773

Test Classification Report:
              precision    recall  f1-score   support

  hatespeech       0.84      0.59      0.70       628
      normal       0.50      0.84      0.63       628
   offensive       0.60      0.37      0.45       628

    accuracy                           0.60      1884
   macro avg       0.65      0.60      0.59      1884
weighted avg       0.65      0.60      0.59      1884



In [7]:
# with explainations 3 epochs
main()


Splitting training dataset...
Initializing BERT model and tokenizer...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\MachineLearning\AICU\recommendation_system\.venv\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



Epoch 1/3
Step 50 - Average training loss: 0.8845
Step 100 - Average training loss: 0.7942
Average training loss: 0.7763


Validation: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:09<00:00,  3.08it/s]


Average validation loss: 0.6443
Validation accuracy: 0.7500

Classification Report:
              precision    recall  f1-score   support

  hatespeech       0.79      0.80      0.80       682
      normal       0.74      0.90      0.81       857
   offensive       0.52      0.13      0.20       261

    accuracy                           0.75      1800
   macro avg       0.68      0.61      0.60      1800
weighted avg       0.73      0.75      0.72      1800

New best model saved with accuracy: 0.7500

Epoch 2/3
Step 50 - Average training loss: 0.5991
Step 100 - Average training loss: 0.5844
Average training loss: 0.5751


...

Average validation loss: 0.5825
Validation accuracy: 0.7683

Classification Report:
              precision    recall  f1-score   support

  hatespeech       0.80      0.84      0.82       682
      normal       0.81      0.86      0.83       857
   offensive       0.43      0.29      0.35       261

    accuracy                           0.77      1800
   macro avg       0.68      0.66      0.67      1800
weighted avg       0.75      0.77      0.76      1800

New best model saved with accuracy: 0.7683

Epoch 3/3
Step 50 - Average training loss: 0.4721
Step 100 - Average training loss: 0.4923
Average training loss: 0.4885


Validation: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:09<00:00,  3.02it/s]


Average validation loss: 0.5793
Validation accuracy: 0.7683

Classification Report:
              precision    recall  f1-score   support

  hatespeech       0.79      0.86      0.82       682
      normal       0.82      0.83      0.83       857
   offensive       0.45      0.32      0.37       261

    accuracy                           0.77      1800
   macro avg       0.68      0.67      0.67      1800
weighted avg       0.75      0.77      0.76      1800


Evaluating model on the test dataset...


Testing: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:10<00:00,  2.99it/s]


Average test loss: 1.2974

Test Classification Report:
              precision    recall  f1-score   support

  hatespeech       0.79      0.63      0.70       628
      normal       0.47      0.89      0.62       628
   offensive       0.67      0.21      0.32       628

    accuracy                           0.58      1884
   macro avg       0.65      0.58      0.55      1884
weighted avg       0.65      0.58      0.55      1884



In [8]:
from sklearn.metrics import classification_report
import torch
import numpy as np
from torch.utils.data import DataLoader

# Create a function to generate the classification report using the saved model
def generate_test_report(test_df, tokenizer, model, le, device):
    # Preprocess test data
    test_texts = test_df['tweet_text']
    test_labels = test_df['label']
    
    # Create test dataset
    test_dataset = CustomDataset(test_texts, test_labels, tokenizer)
    test_loader = DataLoader(test_dataset, batch_size=24)

    # Move model to GPU if available
    model.to(device)
    model.eval()

    all_predictions = []
    all_true_labels = []

    with torch.no_grad():
        for batch in tqdm(test_loader, desc='Testing'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )

            predictions = torch.argmax(outputs.logits, dim=1)
            all_predictions.extend(predictions.cpu().numpy())
            all_true_labels.extend(labels.cpu().numpy())

    # Generate classification report
    print("\nTest Classification Report:")
    print(classification_report(all_true_labels, all_predictions, target_names=le.classes_))

def main():
    try:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        # Load the saved model
        model = BertForSequenceClassification.from_pretrained('fine_tuned_bert')
        tokenizer = BertTokenizer.from_pretrained('fine_tuned_bert')

        # Load the label encoder
        le = np.load('label_classes.npy', allow_pickle=True)

        # Load TEST_DF (ensure it's already preprocessed as mentioned earlier)
        test_df = TEST_DF
        test_df = test_df.dropna(subset=['tweet_text', 'label'])
        test_df = test_df.reset_index(drop=True)
        test_df['label'] = le.transform(test_df['label'])

        # Generate the test report
        generate_test_report(test_df, tokenizer, model, le, device)

    except Exception as e:
        print(f"\nError occurred: {str(e)}")
        print("\nFull error details:")
        import traceback
        traceback.print_exc()



In [9]:
main()


Error occurred: 'numpy.ndarray' object has no attribute 'transform'

Full error details:


Traceback (most recent call last):
  File "C:\Users\usama\AppData\Local\Temp\ipykernel_34924\4269899375.py", line 58, in main
    test_df['label'] = le.transform(test_df['label'])
                       ^^^^^^^^^^^^
AttributeError: 'numpy.ndarray' object has no attribute 'transform'. Did you mean: 'transpose'?
